# Python模拟鼠标动作抓取网页数据


作者：上海交通大学 安泰经济与管理学院 周志中

2017-06-10, updated on 2018-10-20


## 抓取东方财富网基金排名数据

本文介绍如何使用Python+Selenium模拟鼠标点击动作来抓取网页数据。Python+Selenium的文档说明可以在[Selenium with Python](http://selenium-python.readthedocs.io/index.html)找到。制作本文档使用了[Jupyter Notebook](http://jupyter.org/)，可参看[Jupyter Notebook入门（上）](http://codingpy.com/article/getting-started-with-jupyter-notebook-part-1/)和[Jupyter Notebook入门（下）](http://codingpy.com/article/getting-started-with-jupyter-notebook-part-2/)。当中的Markdown语法介绍可在[Markdown 语法说明](http://www.appinn.com/markdown/)和[Markdown——入门指南](http://www.jianshu.com/p/1e402922ee32/)找到。

我们首先抓取的数据是东方财富网提供的股票型的开放式基金[排名情况](http://fund.eastmoney.com/data/fundranking.html)。输入网页网址之后，我们看到的是全部基金排名，如果想看股票型基金排名，还需要点击“股票型”Tab选项卡。这样我们就需要模拟鼠标点击动作，才可以拿到股票型基金排名的数据。不过在此之前我们需要准确定位“股票型”选项卡。在Firefox浏览器中打开网页，使用右键点击选项卡，然后点击Inspect Element（或者按“Q”键），这时候弹出Inspector窗口（见下图）。

![定位“股票型”选项卡的CSS Selector地址](/tree/Pictures/WebScrapper01.jpg)

在Inspector窗口定位到“股票型”选项卡，这时候选项卡整个变成蓝色（你可能需要在Insepector窗口中上下移动1-2行代码）。接下来是在代码行点击右键，然后选择Copy，CSS Selector，将CSS Selector定位地址拷贝到剪贴板。这个定位地址是：#types > li:nth-child(2)。

In [28]:
from selenium import webdriver
import time

driver = webdriver.Firefox();
driver.get("http://fund.eastmoney.com/data/fundranking.html");
          
#Click '股票型'#
driver.find_element_by_css_selector("#types > li:nth-child(2)").click();
time.sleep(10) #sleep 10 seconds till the page is loaded#

上面的语句导入Selenium Webdriver驱动Firefox浏览器，并打开东方财富网的开放式基金排名网页，然后是点击“股票型”选项卡。模拟鼠标点击动作时我们使用了Css Selector定位选项卡元素所在位置，然后使用click()函数对该元素执行点击动作。最后是等待10秒，以便股票型基金的网页数据全部调出来。

![定位表格内容tbody的CSS Selector地址](/tree/Pictures/WebScrapper02.jpg)

我们可以定位到表格的tbody地址（#dbtable > tbody），直接把表格内容全部提取出来，不过更精准的数据提取方式还是在表格当中逐行提取。在Firefox浏览器中表格头中某个元素（例如“基金代码”）右键点击，然后点击Inspect Element（或者按“Q”键），进入Insepector窗口。在定位到的代码行上下几行寻找`<tbody>`下的`<tr>`（table row）标签，然后点击该tag，这时候浏览器中的表格某一行会被选中（见图2）。然后在`<tr>`代码行上点击右键，然后选择Copy，XPath，将XPath定位地址拷贝到剪贴板。分析XPath发现，表格当中某一行的XPath是`/html/body/div[7]/div[3]/table[2]/tbody/tr[i]`，其中i取值从1到50。这就意味着通过XPath`/html/body/div[7]/div[3]/table[2]/tbody/tr`预计可以提取出表格当中所有行。

In [29]:
#elems = driver.find_elements_by_xpath("/html/body/div[7]/div[3]/table[2]/tbody/tr");
elems = driver.find_elements_by_xpath("//table[2]/tbody/tr"); 
#We shortern the full XPath by using '//' to neglect XPath expression before '//'
#Why don't we use elems = driver.find_elements_by_xpath("//tr"); ?
#It will extract all elements with <tr> tag, Yet len(elems)=69 showing that we extract 19 unwanted Web element. 
len(elems) # We find that using the XPath "//table[2]/tbody/tr" , we may get exactly 50 Web elements. 

50

In [30]:
elems = driver.find_elements_by_xpath("//table[2]/tbody/tr"); 
elems[0].text

'1\n161629\n融通证券分级\n10-22 1.1120 0.5020 10.32% 12.66% -2.46% -4.46% -16.66% -32.98% -36.02% -38.45% -24.66% -51.38% -32.60% 0.12%'

将换行符换成空格。

In [31]:
elems[0].text.replace('\n',' ') #将换行符换成空格。

'1 161629 融通证券分级 10-22 1.1120 0.5020 10.32% 12.66% -2.46% -4.46% -16.66% -32.98% -36.02% -38.45% -24.66% -51.38% -32.60% 0.12%'

网页上可以看到50行，而`elems`有50个元素，可见我们已经提取出所有行的数据。其中`elems[0]`对应的是第一行的数据，`elem[49]`对应的是第50行的数据。分析网页元素发现，每个`tr`标签下还有多个`td`标签存放每一行的多个表格的取值。由于表格当中的缺失值被“---”替代，不存在空的格子，因此我们可以直接读每一行的所有信息而不必去读每一个`td`标签当中的取值。

In [32]:
print(elems[0].text.replace('\n',' ').split(' ')) #将一行数据转换成含有多个元素的列表变量
print(len(elems[0].text.replace('\n',' ').split(' '))) #看列表当中有多少个元素

['1', '161629', '融通证券分级', '10-22', '1.1120', '0.5020', '10.32%', '12.66%', '-2.46%', '-4.46%', '-16.66%', '-32.98%', '-36.02%', '-38.45%', '-24.66%', '-51.38%', '-32.60%', '0.12%']
18


使用和上面类似的方法提取出表头名。

In [33]:
#elem = driver.find_element_by_xpath("/html/body/div[7]/div[3]/table[2]/thead/tr");  #full path
elem = driver.find_element_by_xpath("//table[2]/thead/tr"); 
print(elem.text.replace('\n',' ').split(' ')) #看表头信息，发现第一个'比较'和最后一个'全部'不是我们提取出的数据的表头，
#然后'基金代码'被切割成2块了。
print(elem.text.replace('\n',' ').split(' ')[4:-2]) #提取我们所需要的表头，从下标为4的元素开始（第5个元素），到倒数第3个。
ColumnNames = ['序号','基金代码']+elem.text.replace('\n',' ').split(' ')[4:-2]   #加入'序号'、'基金代码'
print(ColumnNames) 
print(len(ColumnNames)) #检查表头长度是否恰好为18个元素。

['比较', '序号', '基金', '代码', '基金简称', '日期', '单位净值', '累计净值', '日增长率', '近1周', '近1月', '近3月', '近6月', '近1年', '近2年', '近3年', '今年来', '成立来', '自定义', '手续费', '可购', '全部']
['基金简称', '日期', '单位净值', '累计净值', '日增长率', '近1周', '近1月', '近3月', '近6月', '近1年', '近2年', '近3年', '今年来', '成立来', '自定义', '手续费']
['序号', '基金代码', '基金简称', '日期', '单位净值', '累计净值', '日增长率', '近1周', '近1月', '近3月', '近6月', '近1年', '近2年', '近3年', '今年来', '成立来', '自定义', '手续费']
18


下面是提取表格当中所有数据存放在df数据框当中。

In [34]:
import pandas as pd
df = pd.DataFrame(columns = ColumnNames); idx =0;
elems = driver.find_elements_by_xpath("//table[2]/tbody/tr"); 
for elem in elems:
    rowdata = elem.text.replace('\n',' ').split(' ')
    df.loc[idx]=rowdata ; idx += 1
df.drop(['序号','自定义'],axis=1, inplace=True)    #删掉2列。
print(df)

      基金代码    基金简称     日期    单位净值    累计净值    日增长率     近1周     近1月      近3月  \
0   161629  融通证券分级  10-22  1.1120  0.5020  10.32%  12.66%  -2.46%   -4.46%   
1   160419  华安中证全指  10-22  1.1084  0.4443   9.81%  12.24%  -2.65%   -4.61%   
2   161027  富国中证全指  10-22  1.1060  0.4240   9.61%  11.94%  -2.37%   -4.45%   
3   004069  南方中证全指  10-22  0.7057  0.7057   9.53%  11.59%  -3.17%   -4.79%   
4   004070  南方中证全指  10-22  0.7009  0.7009   9.52%  11.57%  -3.22%   -4.90%   
5   501016  国泰中证申万  10-22  0.7299  0.7299   9.35%  11.55%  -2.96%   -4.25%   
6   161720  招商中证全指  10-22  0.8113  0.6019   9.46%  11.47%  -3.24%   -5.22%   
7   160633  鹏华证券分级  10-22  1.0010  0.4260   9.52%  11.47%  -2.82%   -4.61%   
8   502010  易方达证券公  10-22  0.7791     ---   9.45%  11.44%  -3.25%   -5.00%   
9   163113  申万菱信申万  10-22  1.1215  1.5002   9.06%  11.40%  -2.92%   -4.62%   
10  502053  长盛中证证券  10-22  1.0210     ---   9.31%  11.34%  -2.95%   -4.88%   
11  540010  汇丰晋信科技  10-22  1.4770  1.4770   6.31%  11.32%  -1.29

In [35]:
driver.close();

最后我们关闭Selenium Webdriver。

## 自动提取百度搜索结果

我们将使用另外一个Selenium Webdriver：Chrome Webdriver。首先需要安装Google Chrome浏览器，可以从[谷歌Chrome下载页](http://www.google.cn/chrome/browser/desktop/)下载。然后需要安装ChromeDriver，下载页在[Github的ChromeDriver页](https://github.com/SeleniumHQ/selenium/wiki/ChromeDriver)。不过由于Google被Great Firewall封杀，可以从课程文件夹的/Software/Selenium WebDrivers目录下载。使用ChromeDriver之前要确保ChromeDriver.exe所在目录已经加入Path环境变量中，可参考[Win10中添加环境变量](http://jingyan.baidu.com/article/7e44095332c60c2fc1e2ef58.html)。

In [1]:
from selenium import webdriver
import time

#driver = webdriver.Firefox(); #需要确保geckodriver.exe是最新版本可以操作最新版本的Firefox
driver = webdriver.Chrome(); #需要确保chromedriver.exe是最新版本可以操作最新版本的Chrome浏览器
driver.get("https://www.baidu.com");

运行上面的代码可以使用Selenium Chrome Webdriver打开百度网页。我们打算在搜索栏当中输入“上海交通大学”。在Chrome浏览器中点击搜索栏，然后点击鼠标右键，选择“检查”。在右边的HTML代码行中选择对应搜索框的代码行，然后点击右键，选择Copy，Copy selector，这样就把CSS selector复制到剪贴板。这个CSS selector是：#kw。填写完搜索框我们还要点击“百度一下”按钮。在该按钮上点击右键，同样选择Copy，Copy selector，找到它的CSS selector是：#su。

In [2]:
inputbox = driver.find_element_by_css_selector("#kw")
inputbox.clear() #Clear the searching box.
inputbox.send_keys("上海交通大学")
submitbutton= driver.find_element_by_css_selector("#su")
submitbutton.click()

以上代码实现了在搜索栏中填入“上海交通大学”，然后点击“百度一下”按钮。接下来我们希望提取出搜索结果的文本和URL链接。每页有10个搜索结果，通过分析搜索结果的10个元素，发现搜索结果的文本和URL链接都在h3标签下第一个a标签里，转换成xPath是：//h3/a[1]。h3前面的//（两个斜杠）代表xPath前面是任何字符。例如xPath='/html/body/div[1]/div[5]/div[1]/div[3]/div[5]/h3/a[1]'的时候，这个xPath和'//h3/a[1]'也匹配。提取多个网页元素要用到`driver.find_elements_by_xpath`函数。

In [3]:
#Get elements that store text and URL of search results.
search_results=driver.find_elements_by_xpath("//h3/a[1]")

其中搜索结果的文本是`</a>`和`<a>`标签之间的值。而搜索结果的URL链接则是`</a>`标签中`href`属性的取值。

In [4]:
Search_Texts =[];
Search_URLs = [];
for i in range(0,len(search_results)):
    Search_Texts.append(search_results[i].text)
    Search_URLs.append(search_results[i].get_attribute('href'))
print(Search_Texts)
print(Search_URLs[0],Search_URLs[1])

['上海交通大学', '上海交通大学_百度百科', '上海交通大学高考分数线_招生信息_中国教育在线', '--上海交通大学发展规划处--', '上海交通大学研究生院', '研究院-上海交通大学', '上海交通大学本科招生网-交大招生办-首页', '概况-上海交通大学', '上海交通大学吧_百度贴吧', '在上海市搜索上海交通大学_百度地图']
http://www.baidu.com/link?url=HzCMkjsASUVNqGuaoAgNzsausxKATI1RtOcRuBOXIegxFAYdRwxU8COUNcPqrySj http://www.baidu.com/link?url=z_mpU0QD8L02RJyTB4iDq92gRPX0bYHsAzAPOR1hXZcsrhgkAt_l_AQBPBz9hrI3r8T1T2O18LSLQ_JjuNqtw7EW2SncvXYsm--onV1Fa44306p4gnIYjPElQz0tdLGxBIr-PkYYV0fAByjcwS8Kv_


通过上面的代码，我们得到了第一页搜索结果的所有文本和URL链接。接下来点击“下一页”，可以提取第2页的搜索结果。然后继续点击“下一页”，可以提取第3页的搜索结果。注意：在第1页点击“下一页”时，“下一页”按钮的xpath是：`//*[@id='page']/a[10]`。在第2页点击“下一页”时，“下一页”按钮的xpath是：`//*[@id='page']/a[11]`，这是因为第2页多了一个“上一页”的按钮。第3页，第4页等后续页面的“下一页”按钮的xpath仍然是：`//*[@id='page']/a[11]`保持不变。

In [5]:
#click next page to get page 2
driver.find_element_by_xpath("//*[@id='page']/a[10]").click()
time.sleep(10) #sleep 10 seconds till the page is loaded#
#click next page to get page 3
driver.find_element_by_xpath("//*[@id='page']/a[11]").click()

最后我们关闭Selenium Webdriver。

In [6]:
driver.close();

## 使用Selenium IDE录制鼠标动作

如果我们在提取数据前，有多个鼠标动作并且要输入文字（比如搜索关键词），那么一个个去定位网页元素就非常麻烦。这时可以使用Selenium IDE来录制鼠标动作并自动转成Python语句。Selenium IDE是Firefox浏览器的一个扩展组件（Add-on），可在[Mozilla的Selenium IDE Add-on页面](https://addons.mozilla.org/en-US/firefox/addon/selenium-ide/)下载。

Selenium IDE安装完成之后，在Firefox浏览器右上角三条杠附近出现一个新图标（形状是一支笔在一张纸上写字），鼠标移动到该图标上悬浮可见“Selenium IDE”出现。点击这个图标，弹出Selenium IDE窗口。在Selenium IDE窗口有一个红色的录制按钮，点击这个按钮开始录制鼠标动作。

接下来我们在Firefox浏览器中输入`https://jenner.com/people`，选择Locations，然后点击New York，选择Law School，然后下拉菜单直到看到Coloumnbia University School of Law，选择这个法学院，接着点击Search by旁边的Keyword，选择关键词搜索栏，输入Associate，最后点击Submit。这时候Selenium IDE已经完成了对所有鼠标动作的录制工作。回到Selenium IDE窗口，点击红色按钮停止录制。在Selenium IDE窗口点击文件，选择Export Test Case As...，选择Python 2/unittest/WebDriver，将录制好的鼠标动作存到新的py文件当中（假设是Record.py）。使用Spyder或者记事本打开Record.py，可以发现`driver.find_element_by...`语句都是我们需要的。

In [7]:
from selenium import webdriver
import time
driver = webdriver.Firefox();
driver.get("https://jenner.com/people");

上面是使用Firefox WebDriver打开Jenner & Block的人员列表页。接下来是一系列鼠标操作：

In [8]:
driver.find_element_by_xpath("//form[@id='new_search']/div[4]/div/h1").click()
driver.find_element_by_id("search_offices_new_york").click()
driver.find_element_by_css_selector("div.filter.schools > div.header > h1").click()
driver.find_element_by_xpath("//form[@id='new_search']/div[5]/div/h1").click()

上面的鼠标操作已经选择了Locations搜索栏中的New York，然后选择了Law School搜索栏的下拉列表。这里需要强调一下：Selenium IDE只是录制了鼠标点击动作，并没有录制鼠标的scrolling（上下卷）动作。而点击了Law School的下拉列表是看不到“Coloumnbia University School of Law”这个选项的，必须把鼠标下拉一定距离，才能看到“Coloumnbia University School of Law”这个选项。下面的语句就是把鼠标下拉菜单1000个像素，此时“Coloumnbia University School of Law”这个选项会出现。

In [9]:
#scroll down the page till 'Coloumnbia University School of Law' is displayed
driver.execute_script('window.scrollTo(0,1000)') 

再次强调，上面的语句是手工加进去的，而不是Selenium IDE录制好的鼠标动作。如果网页元素在页面上不可见，那么使用`driver.find_element`函数就会出错。我们下拉菜单的目的是确保选项在页面上是可见的。下面的一系列鼠标动作和填写搜索栏的动作都是Selenium IDE录制的结果：

In [10]:
driver.find_element_by_id("search_schools_0075").click()
driver.find_element_by_xpath("(//input[@name='search_scope'])[2]").click()
driver.find_element_by_id("search_full_name").clear()
driver.find_element_by_id("search_full_name").send_keys("Associate")
driver.find_element_by_name("commit").click()

![Jenner & Block的搜索结果](/tree/Pictures/WebScrapper04.png)

上图是搜索结果，我们希望从红色框中提取出搜索结果中的所有人名。通过分析网页元素，可以写下面的代码：

In [11]:
Search_Result = driver.find_element_by_css_selector("#searchresultsbox > \
div.column.result-container > div.people-search-results > div.name-row")
Search_Result.text.split('\n')

['Brittany R. Lamb',
 'Justin O. Spiegel',
 'Gretchen O. Stertz',
 'Irene Ten Cate',
 'Bernadette M. Walli']

大功告成，关闭Selenium Webdriver。

In [12]:
driver.close();

其他有用的资料有：Python学习指南中的[Python爬虫](http://www.cnblogs.com/miqi1992/p/8082471.html)。